In [131]:
%matplotlib inline

In [133]:
import csv
import pandas as pd

In [135]:
# Read csv file
file = 'IMDb_All_Genres_etf_clean1.csv'
df = pd.read_csv(file)
df

,Movie_Title,Year,Director,Actors,Rating,Runtime(Mins),Censor,Total_Gross,main_genre,side_genre
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,148,UA,Gross Unkown,Action,"Adventure, Drama"
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,152,UA,$534.86M,Action,"Crime, Drama"
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,201,U,$377.85M,Action,"Adventure, Drama"
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,148,UA,$292.58M,Action,"Adventure, Sci-Fi"
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,179,UA,$342.55M,Action,"Adventure, Drama"
...,...,...,...,...,...,...,...,...,...,...
5557,Disaster Movie,2008,"Directors:Jason Friedberg, Aaron Seltzer","Carmen Electra, Vanessa Lachey, Nicole Parker,...",1.9,87,PG-13,$14.19M,Comedy,Sci-Fi
5558,The Hottie & the Nottie,2008,Tom Putnam,"Paris Hilton, Joel David Moore, Christine Laki...",1.9,91,PG-13,$0.03M,Comedy,Romance
5559,From Justin to Kelly,2003,Robert Iscove,"Kelly Clarkson, Justin Guarini, Katherine Bail...",1.9,81,PG,$4.92M,Comedy,"Musical, Romance"
5560,Superbabies: Baby Geniuses 2,2004,Bob Clark,"Jon Voight, Scott Baio, Vanessa Angel, Skyler ...",1.5,88,PG,$9.11M,Comedy,"Family, Sci-Fi"


In [137]:
# The movie for which you want to find similar movies
target_movie = 'The Dark Knight' 

In [139]:
from sklearn.metrics.pairwise import pairwise_distances

In [141]:
# Calculate pairwise Euclidean distance between movies based on their ratings
ratings = df['Rating'].values.reshape(-1, 1)
euclidean_distances = pairwise_distances(ratings, metric='euclidean')

# Convert the Euclidean distance matrix into a DataFrame 
euclidean_distances_df = pd.DataFrame(euclidean_distances, index=df['Movie_Title'], columns=df['Movie_Title'])

# Get the distance scores for the target movie
distances = euclidean_distances_df[target_movie].sort_values(ascending=True)

# Exclude the target movie from the top 10 list of similar movies
similar_movies = distances.drop(target_movie)

# Print the top 10 similar movies
top_similar_movies = similar_movies.head(10)
print(top_similar_movies)

Movie_Title
12 Angry Men                                     0.0
The Lord of the Rings: The Return of the King    0.0
Schindler's List                                 0.0
Aynabaji                                         0.0
The Godfather Part II                            0.0
777 Charlie                                      0.1
Rocketry: The Nambi Effect                       0.1
Jai Bhim                                         0.1
Pulp Fiction                                     0.1
Hababam Sinifi                                   0.2
Name: The Dark Knight, dtype: float64


In [142]:
# Select relevant columns: Movie titles and their rating
ratings_df = df[['Movie_Title', 'Rating']]

In [145]:
# Extract movie ratings
ratings = ratings_df['Rating'].values.reshape(-1, 1)

In [147]:
from sklearn.cluster import KMeans

In [149]:
# Set k-value
k = 12

In [151]:
cluster_model = KMeans(n_clusters=k)

In [153]:
cluster_model.fit(ratings)

/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=12)

In [155]:
cluster_labels = cluster_model.predict(ratings)
rating_cluster_df = pd.DataFrame(cluster_labels, index=df.index, columns=["cluster"])

In [157]:
# Display the number of items in each cluster
rating_cluster_df["cluster"].value_counts()

cluster
3     901
6     779
11    728
2     673
5     656
10    504
0     482
1     466
8     167
9     141
4      42
7      23
Name: count, dtype: int64

In [175]:
# Group similar ratings in clusters
for cluster,ratings in rating_cluster_df.groupby("cluster"):
    print("Cluster:", cluster, "Size:", ratings.shape[0])
    
    for r_id in ratings.sample(5).index:
        print("\t", r_id, ratings_df.loc[r_id])

Cluster: 0 Size: 482
	 1456 Movie_Title    Baywatch
Rating              5.5
Name: 1456, dtype: object
	 5456 Movie_Title    Scary Movie 2
Rating                   5.3
Name: 5456, dtype: object
	 5370 Movie_Title    Valentine's Day
Rating                     5.7
Name: 5370, dtype: object
	 5437 Movie_Title    Christmas with the Kranks
Rating                               5.4
Name: 5437, dtype: object
	 2851 Movie_Title    Enough
Rating            5.7
Name: 2851, dtype: object
Cluster: 1 Size: 466
	 88 Movie_Title    Nefes: Vatan Sagolsun
Rating                           8.0
Name: 88, dtype: object
	 3018 Movie_Title    Harry Potter and the Prisoner of Azkaban
Rating                                              7.9
Name: 3018, dtype: object
	 3644 Movie_Title    Boogie Nights
Rating                   7.9
Name: 3644, dtype: object
	 3664 Movie_Title    The Lost Weekend
Rating                      7.9
Name: 3664, dtype: object
	 3003 Movie_Title    Kahaani
Rating             8.1
Name: 3003